# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders = pd.read_csv(r'C:\Users\CLAUDIOCLEMENTE\Bootcamp\dataV2-labs\module-1\Dataframe-Calculations\Orders.csv')

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [5]:
# your code here
cust_spend = orders.groupby(["CustomerID"])[["amount_spent"]].agg("sum")
cust_spend.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [6]:
quantile_labels = ['Small', 'Medium', 'Large', 'Preferred Cust.', 'VIP']
cust_spend['category'] = pd.qcut(cust_spend['amount_spent'],q=[0, .2, .4, .6, .8, 1],labels=quantile_labels)
cust_spend.head()

,amount_spent,category
CustomerID,,
12346,77183.60,VIP
12347,4310.00,VIP
12348,1797.24,Preferred Cust.
12349,1757.55,Preferred Cust.
12350,334.40,Medium


In [7]:
orders = orders.merge(cust_spend["category"],on="CustomerID")
orders.head()
x = orders.loc[orders["Country"] == 'Canada']


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here
cust_country_spend = orders.groupby(["Country","category"])[["CustomerID"]].agg("count").reset_index()
cust_country_spend.head(60)

top_country = cust_country_spend.loc[cust_country_spend["category"] == 'VIP']

top_country.sort_values(by="CustomerID",ascending=False)

,Country,category,CustomerID
179,United Kingdom,VIP,198270.0
54,EIRE,VIP,7238.0
74,Germany,VIP,6806.0
69,France,VIP,6112.0
119,Netherlands,VIP,2080.0
154,Spain,VIP,1569.0
164,Switzerland,VIP,1307.0
19,Belgium,VIP,1256.0
134,Portugal,VIP,1024.0
4,Australia,VIP,998.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [28]:
# your code here

cust_country_spend = orders.groupby(["Country","category"])[["CustomerID"]].agg("count").reset_index()
cust_country_spend.head(60)

top_country = cust_country_spend.loc[(cust_country_spend.category=="VIP") | (cust_country_spend.category=="Preferred Cust.")]

top_top_country = top_country.groupby(["Country"])[["CustomerID"]].agg("sum").reset_index()

top_top_country.head()
top_top_country.sort_values(by="CustomerID",ascending=False)

,Country,CustomerID
35,United Kingdom,274276.0
14,Germany,7928.0
10,EIRE,7238.0
13,France,7014.0
23,Netherlands,2241.0
30,Spain,2183.0
3,Belgium,1834.0
32,Switzerland,1672.0
26,Portugal,1093.0
0,Australia,1092.0
